# Comparison of reference Happy Jack results to Snotel Data

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

__Read in tRIBS element/pixel file results__\
Note: you can modify ```path_to_elem``` for updated model simulation results.

In [ ]:
# for comparison to reference simulation update the line below to  '../results/reference/hj_ref0.pixel'
path_to_elem = '../results/test/hj_test0.pixel'
elem = pd.read_csv(path_to_elem, sep=r"\s+", header=0)

# convert model hourly time step to dates
startdate = "06/01/2002/00/00"
month = int(startdate[0:2])
day = int(startdate[3:5])
year = int(startdate[6:10])
minute = int(startdate[11:13])
second = int(startdate[14:16])
date = pd.Timestamp(year=year, month=month, day=day, minute=minute)
dt = pd.to_timedelta(elem['Time_hr'], unit='h')
elem['date'] = [date + step for step in dt]
elem.set_index('date',inplace=True)

__Read in the snotel data__\
Snotel data is for the Happy Jack site in Northern Arizona. Data is from:
>
Sun N, Yan H, Wigmosta MS, Leung LR, Skaggs R, Hou Z. 2019. Regional snow parameters estimation for large-domain hydrological applications in the Western United States. Journal of Geophysical Research: Atmospheres 124 : 5296–5313.
>

In [ ]:
# Specify the path to snotel data
file_path = '../data/Snotel/bcqc_34.75000_-111.41000.txt'

# precip is in inches, T is temperature in F, and SWE is snow water equivalent in inches
column_names = ['year', 'month', 'day', 'precip', 'maxT','minT','meanT','swe']
snotel = df = pd.read_csv(file_path,  sep='\s+', header=None, na_values=['nan'], names=column_names)

#convert year, month, day to datetime and drop columns
snotel['date'] = pd.to_datetime(snotel [['year', 'month', 'day']])
snotel  = snotel .drop(['year', 'month', 'day'], axis=1)
snotel.set_index('date',inplace=True)

#convert swe to cm 
snotel['swe_cm'] = snotel['swe'] * 2.54
snotel['precip'] = snotel['precip']*25.4 #mm

__Prep Data For Comparions Of Peak SWE with SNOTEL Site__

In [ ]:
# comparison of magnitude and timing of peak swe

# create column for water year
snotel['WaterYear'] = snotel.index.year - (snotel.index.month < 10)
elem['WaterYear'] = elem.index.year - (elem.index.month < 10)

# create a dataframe with index of water year, and columns of for peak swe and date of peak swe
peak_swe_snotel = snotel.groupby('WaterYear')['swe_cm'].agg([('PeakSWE', 'max'), ('PeakDate', 'idxmax')])
peak_swe_elem = elem.groupby('WaterYear')['SnWE_cm'].agg([('PeakSWE', 'max'), ('PeakDate', 'idxmax')])

# overlapping water years
start_year = 2002
end_year = 2017

# subset dataframes based on overlap
peak_swe_snotel = peak_swe_snotel[(peak_swe_snotel.index >= start_year) & (peak_swe_snotel.index <= end_year)]
peak_swe_elem = peak_swe_elem[(peak_swe_elem.index >= start_year) & (peak_swe_elem.index <= end_year)]

# calculate the difference in peak dates
peak_date_diff = (peak_swe_snotel['PeakDate'] - peak_swe_elem['PeakDate']).dt.days


__Compare time series and peak swe__

In [ ]:
cm = 1/2.54  # centimeters in inches
plt.rc({'size': 14})
mpl.rcParams['pdf.fonttype'] = 42
fig,ax = plt.subplots(2,1,figsize=[18*cm,12*cm],gridspec_kw={'height_ratios': [1, 3]},layout='constrained')

# time series plot of SWE
ax[0].plot(snotel.index,snotel.swe_cm,label = 'SNOTEL',alpha =0.6,c='xkcd:black',linewidth=1.5)
ax[0].plot(elem.index,elem.SnWE_cm,label = 'tRIBS',alpha =0.5,c='xkcd:blue',linewidth=1.5)
ax[0].set_xlim([min(elem.index),max(elem.index)])
ax[0].set_ylabel('SWE (cm)')
ax[0].set_xlabel('Date')
ax[0].legend(loc='upper left',frameon=False)


# plot comparsion
ax[1].plot([0,50],[0,50],c='k',linestyle='--')
mp = ax[1].scatter(peak_swe_elem['PeakSWE'], peak_swe_snotel['PeakSWE'],s=100, c=peak_date_diff, cmap='viridis')

# color bar location 
cax = ax[1].inset_axes([25, 10., 30, 2.5], transform=ax[1].transData)
fig.colorbar(mp, cax=cax, orientation='horizontal',label='Difference in days of peak SWE (SNOTEL - tRIBS)')

ax[1].set_xlabel('Peak SWE (cm) : tRIBS')
ax[1].set_ylabel('Peak SWE (cm) : SNOTEL')


#plt.savefig('HJ_SWE_time_series.pdf',dpi=600)